In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import util
from scipy import stats
import os
import regex as re

In [14]:
parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [15]:
pd.set_option('max_columns', None)

In [16]:
ksi_df = pd.read_csv(parent + '\Data\KSI.csv')
clustered_df = pd.read_csv(parent + '\Data\Created datasets\Clustered_KSI.csv', index_col = 0)

In [17]:
clustered_df =  clustered_df['Cluster']
ksi_df = pd.concat([ksi_df, clustered_df], axis = 1)

In [18]:
# I AM ONLY LOOKING AT AGE AND BEHAVIOUR FOR DRIVERS NOT DRIVING PROPERLY
# You may ask - What about drivers who were driving properly in normal condition, but still had an accident?
# As long as DRIVACT != 'Driving Properly', the driver action was wrong somehow
# IF THEY had an accident because of environmental factors, DRIVACT will be 'Lost Control'
# NEGLECTING PASSENGERS, PEDESTRIANS, ETC.
# this should give about 4.7k records

ksi_df = ksi_df[(ksi_df['INVTYPE'].str.contains('Driver', flags = re.IGNORECASE)) & ~(ksi_df['DRIVACT'].str.contains('Driving Properly', flags = re.IGNORECASE))]


# Use this if you also want to include pedestrian and cyclist actions
# But this leads to MANY complications
# ksi_df = ksi_df[(ksi_df['INVTYPE'].str.contains('Driver', flags = re.IGNORECASE)) | ksi_df['IMPACTYPE'].str.contains('Pedestrian | Cyclist', flags = re.IGNORECASE, regex = True)]

In [19]:
ksi_df.shape

(4706, 58)

In [20]:
ksi_df['LATITUDE'] = ksi_df['LATITUDE'].apply(util.truncate, args = (3, ))
ksi_df['LONGITUDE'] = ksi_df['LONGITUDE'].apply(util.truncate, args = (3, ))

In [21]:
ksi_df

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,DIVISION,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,ObjectId,Cluster
0,-8.844611e+06,5.412414e+06,3387730,892658,2006,2006/03/11 05:00:00+00,852,8,BLOOR ST W,DUNDAS ST W,<Null>,Major Arterial,Toronto and East York,4,11,43.656,-79.452,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Pedestrian Collisions,Driver,unknown,None,<Null>,South,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,D11,88,High Park North (88),1,5.0
2,-8.816480e+06,5.434843e+06,3388101,892810,2006,2006/03/11 05:00:00+00,915,9,MORNINGSIDE AVE,SHEPPARD AVE E,<Null>,Major Arterial,Scarborough,25,42,43.801,-79.199,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Fatal,Turning Movement,Motorcycle Driver,45 to 49,Fatal,<Null>,East,Motorcycle,Turning Right,Disobeyed Traffic Control,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D42,131,Rouge (131),3,6.0
4,-8.822759e+06,5.424516e+06,3387793,892682,2006,2006/03/12 05:00:00+00,240,2,EGLINTON AVE E,COMMONWEALTH AVE,<Null>,Major Arterial,Scarborough,"21,20",41,43.734,-79.256,Mid-Block,<Null>,No Control,Clear,Dark,Dry,Fatal,Pedestrian Collisions,Driver,25 to 29,None,<Null>,West,"Automobile, Station Wagon",Going Ahead,Other,"Ability Impaired, Alcohol",<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,D41,138,Eglinton East (138),5,3.0
7,-8.841082e+06,5.411337e+06,3388371,892913,2006,2006/03/12 05:00:00+00,1903,19,OSSINGTON AVE,DUNDAS ST W,<Null>,Major Arterial,Toronto and East York,"9,11,10",14,43.649,-79.420,Intersection,At Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Driver,30 to 34,None,<Null>,East,"Automobile, Station Wagon",Turning Left,Failed to Yield Right of Way,"Ability Impaired, Alcohol Over .08",<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,D14,81,Trinity-Bellwoods (81),8,5.0
19,-8.829249e+06,5.437050e+06,3398180,896606,2006,2006/03/16 05:00:00+00,925,9,BIRCHMOUNT RD,SANWOOD BLVD,<Null>,Major Arterial,Scarborough,22,42,43.816,-79.314,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Turning Movement,Driver,25 to 29,Major,<Null>,North,"Automobile, Station Wagon",Going Ahead,Disobeyed Traffic Control,Normal,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,Yes,<Null>,<Null>,D42,116,Steeles (116),20,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16850,-8.832251e+06,5.411934e+06,81509540,1604895,2020,2020/08/25 04:00:00+00,2148,21,GARDINER XWY E,LAKE SHORE BLVD E OFF RAMP,65 m West of,<Null>,<Null>,<Null>,<Null>,43.653,-79.341,Mid-Block,Non Intersection,No Control,Clear,"Dark, artificial",Dry,Non-Fatal Injury,Rear End,Motorcycle Driver,40 to 44,Major,<Null>,East,Motorcycle,Going Ahead,Speed too Fast For Condition,Inattentive,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,D55,70,South Riverdale (70),16851,0.0
16852,-8.839890e+06,5.428103e+06,81506508,1630486,2020,2020/08/29 04:00:00+00,1419,14,YONGE ST,FLORENCE AV,

In [31]:
# dont run this here, it just makes life harder

#ksi_df.replace('<Null>', np.nan, inplace = True)

In [22]:
# Pairwise mode
# https://stackoverflow.com/questions/55719762/how-to-calculate-mode-over-two-columns-in-a-python-dataframe

cols = ['INVAGE', 'RDSFCOND', 'VISIBILITY', 'LIGHT', 'TRAFFCTL', 'DRIVACT', 'DRIVCOND', 'VEHTYPE', 'Cluster']

ksi_df['COMBINED'] = ksi_df[cols].apply(tuple, axis = 1)

In [23]:
# checking results

ksi_df['COMBINED'].head(2)

0    (unknown, Dry, Clear, Daylight, Traffic Signal...
2    (45 to 49, Dry, Clear, Daylight, Traffic Signa...
Name: COMBINED, dtype: object

In [24]:
# OLD CODE - DO NOT RUN ANYMORE
# its ugly, its stupid, buts its mine, and it works

# it is our assumption that environmental and behavioural conditions are correlated - thus they occur together
# therefore, this is akin to taking 'global mode'

# l = []
# for col in cols:    
#     l.append(ksi_df.groupby(['LATITUDE', 'LONGITUDE'])[col].apply(lambda x: x.mode().iloc[0]).reset_index())

# who cares about vectorization?
# loop like a neanderthal

# looper = l[0]
# for df in range(1, len(l)):
#     looper = looper.merge(l[df], on = ['LATITUDE', 'LONGITUDE'], how = 'inner')

# The issue with this code is the fact that column wise modes are being taken. This MIGHT lead to nonsensical results
# In the improved code - modes are being taken ACROSS column values.

In [35]:
ksi_df[(ksi_df['LATITUDE'] == 43.592) & (ksi_df['LONGITUDE'] == -79.542)]

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,DIVISION,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,ObjectId,COMBINED
13662,-8.854584e+06,5.402622e+06,80609330,7000125598,2017,2017/01/21 05:00:00+00,40,0,BROWNS LINE,LAKESHORE W BROWNS LI RAMP,20 m North,Major Arterial,Etobicoke York,3,22,43.592,-79.542,Mid-Block,Non Intersection,No Control,"Fog, Mist, Smoke, Dust","Dark, artificial",Wet,Fatal,SMV Other,Driver,20 to 24,Fatal,3,South,Delivery Van,Going Ahead,Lost control,Had Been Drinking,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,D22,19,Long Branch (19),13663,"(20 to 24, Wet, Fog, Mist, Smoke, Dust, Dark, ..."


In [25]:
# level_2 indicates # of modes returned
# important because multiple drivers may have been at fault

feature_collapse = ksi_df.groupby(['LATITUDE', 'LONGITUDE'])['COMBINED'].apply(lambda x: x.mode()).reset_index()

feature_collapse

,LATITUDE,LONGITUDE,level_2,COMBINED
0,43.592,-79.542,0,"(20 to 24, Wet, Fog, Mist, Smoke, Dust, Dark, ..."
1,43.592,-79.541,0,"(15 to 19, Dry, Clear, Dark, No Control, Impro..."
2,43.592,-79.534,0,"(35 to 39, Dry, Clear, Daylight, Stop Sign, Fa..."
3,43.593,-79.541,0,"(40 to 44, Wet, Clear, Dark, artificial, No Co..."
4,43.593,-79.538,0,"(25 to 29, Dry, Clear, Daylight, Traffic Signa..."
...,...,...,...,...
4673,43.853,-79.179,1,"(55 to 59, Dry, Clear, Daylight, Stop Sign, Fa..."
4674,43.854,-79.175,0,"(20 to 24, Dry, Clear, Daylight, No Control, O..."
4675,43.854,-79.169,0,"(15 to 19, Dry, Clear, Dark, No Control, Lost ..."
4676,43.855,-79.171,0,"(25 to 29, Dry, Clear, Dark, No Control, Lost ..."


In [37]:
# interesting accidents occurred at this lat and long - bay street
# most recent accident occurred in 2019

ksi_df[(ksi_df['LATITUDE'] == 43.650) & (ksi_df['LONGITUDE'] == -79.386)]

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,HOUR,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,DIVISION,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,POLICE_DIVISION,HOOD_ID,NEIGHBOURHOOD,ObjectId,COMBINED
4191,-8.837275e+06,5.411568e+06,5807574,1107544,2009,2009/06/02 04:00:00+00,2013,20,QUEEN St W,UNIVERSITY Ave,<Null>,Major Arterial,Toronto and East York,10,52,43.65,-79.386,Intersection,At Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Driver,20 to 24,Minimal,<Null>,West,"Automobile, Station Wagon",Going Ahead,Disobeyed Traffic Control,Unknown,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,Yes,<Null>,<Null>,D52,76,Bay Street Corridor (76),4192,"(20 to 24, Dry, Clear, Dark, Traffic Signal, D..."
5035,-8.837241e+06,5.411445e+06,5303803,1133583,2009,2009/10/04 04:00:00+00,1400,14,UNIVERSITY Ave,RICHMOND St W,<Null>,Major Arterial,Toronto and East York,10,52,43.65,-79.386,Intersection,At Intersection,Traffic Signal,Rain,Daylight,Wet,Non-Fatal Injury,Angle,Driver,55 to 59,None,<Null>,North,"Automobile, Station Wagon",Going Ahead,Disobeyed Traffic Control,Inattentive,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,Yes,<Null>,<Null>,D52,76,Bay Street Corridor (76),5036,"(55 to 59, Wet, Rain, Daylight, Traffic Signal..."
6144,-8.837275e+06,5.411568e+06,6183729,1186370,2010,2010/07/18 04:00:00+00,427,4,QUEEN ST W,UNIVERSITY AVE,<Null>,Major Arterial,Toronto and East York,10,52,43.65,-79.386,Intersection,<Null>,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Pedestrian Collisions,Driver,35 to 39,None,<Null>,East,"Automobile, Station Wagon",Going Ahead,Disobeyed Traffic Control,Inattentive,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D52,76,Bay Street Corridor (76),6145,"(35 to 39, Dry, Clear, Dark, Traffic Signal, D..."
6177,-8.837275e+06,5.411568e+06,6209549,1197349,2010,2010/07/29 04:00:00+00,905,9,UNIVERSITY AVE,QUEEN ST W,<Null>,Major Arterial,Toronto and East York,10,52,43.65,-79.386,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Cyclist Collisions,Driver,50 to 54,None,<Null>,North,"Automobile, Station Wagon",Going Ahead,Disobeyed Traffic Control,Inattentive,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D52,76,Bay Street Corridor (76),6178,"(50 to 54, Dry, Clear, Daylight, Traffic Signa..."
6413,-8.837275e+06,5.411568e+06,6617310,1229063,2011,2011/03/18 04:00:00+00,130,1,UNIVERSITY Aven,QUEEN Stre W,<Null>,Major Arterial,Toronto and East York,10,52,43.65,-79.386,Intersection,At Intersection,Traffic Signal,Clear,Dark,Dry,Non-Fatal Injury,Angle,Driver,45 to 49,Major,<Null>,North,"Automobile, Station Wagon",Going Ahead,Disobeyed Traffic Control,Inattentive,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D52,76,Bay Street Corridor (76),6414,"(45 to 49, Dry, Clear, Dark, Traffic Signal, D..."
7305,-8.837241e+06,5.411445e+06,7399367,1287560,2012,2012/01/31 05:00:00+00,1525,15,UNIVERSITY AVE,RICHMOND ST W,<Null>,Major Arterial,Toronto and East York,10,52,43.65,-79.386,Intersection,At Intersection,Traffic Signal,Clear,Daylight,Dry,Non-Fatal Injury,Angle,Driver,40 to 44,Major,<Null>,North,"Automobile, Station Wagon",Going Ahead,Disobeyed Traffic Control,Inattentive,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,<Null>,<Null>,<Null>,<Null>,<Null>,<Null>,Yes,Yes,<Null>,<Null>,D52,76,Bay Street Corridor (76),7306,"(40 to 44, Dry, Clear, Daylight, Traffi

In [26]:
time_df = ksi_df.groupby(['LATITUDE', 'LONGITUDE'])['TIME'].apply(lambda x: x.mean()).reset_index()

In [27]:
# Unpacking tuple into columns

feature_collapse = feature_collapse.join(feature_collapse['COMBINED'].apply(pd.Series, index = cols))
feature_collapse = feature_collapse.merge(time_df, on = ['LATITUDE', 'LONGITUDE'], how = 'inner')

# Dropping level_2 and COMBINED since they have served their purpose

feature_collapse.drop(['level_2', 'COMBINED'], axis = 1, inplace = True)

In [28]:
feature_collapse

,LATITUDE,LONGITUDE,INVAGE,RDSFCOND,VISIBILITY,LIGHT,TRAFFCTL,DRIVACT,DRIVCOND,VEHTYPE,Cluster,TIME
0,43.592,-79.542,20 to 24,Wet,"Fog, Mist, Smoke, Dust","Dark, artificial",No Control,Lost control,Had Been Drinking,Delivery Van,9.0,40.000000
1,43.592,-79.541,15 to 19,Dry,Clear,Dark,No Control,Improper Turn,Normal,"Automobile, Station Wagon",8.0,46.000000
2,43.592,-79.534,35 to 39,Dry,Clear,Daylight,Stop Sign,Failed to Yield Right of Way,Inattentive,"Automobile, Station Wagon",2.0,1252.000000
3,43.593,-79.541,40 to 44,Wet,Clear,"Dark, artificial",No Control,Speed too Fast For Condition,Unknown,"Automobile, Station Wagon",9.0,520.000000
4,43.593,-79.538,25 to 29,Dry,Clear,Daylight,Traffic Signal,Improper Turn,Normal,"Automobile, Station Wagon",1.0,1647.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
4673,43.853,-79.179,55 to 59,Dry,Clear,Daylight,Stop Sign,Failed to Yield Right of Way,Inattentive,"Automobile, Station Wagon",2.0,1224.500000
4674,43.854,-79.175,20 to 24,Dry,Clear,Daylight,No Control,Other,Fatigue,"Automobile, Station Wagon",0.0,1645.000000
4675,43.854,-79.169,15 to 19,Dry,Clear,Dark,No Control,Lost control,Inattentive,"Automobile, Station Wagon",9.0,2140.000000
4676,43.855,-79.171,25 to 29,Dry,Clear,Dark,No Control,Lost control,Normal,Motorcycle,3.0,16.000000


In [41]:
# further inspection reveals actual nulls in environmental variables
# need to treat these separately when performing regression

feature_collapse[feature_collapse['VISIBILITY'] != 'Clear']

,LATITUDE,LONGITUDE,INVAGE,RDSFCOND,VISIBILITY,LIGHT,TRAFFCTL,DRIVACT,DRIVCOND,VEHTYPE,TIME
0,43.592,-79.542,20 to 24,Wet,"Fog, Mist, Smoke, Dust","Dark, artificial",No Control,Lost control,Had Been Drinking,Delivery Van,40.000000
9,43.594,-79.534,60 to 64,Wet,Rain,Dark,No Control,Lost control,"Ability Impaired, Alcohol Over .08","Automobile, Station Wagon",1163.000000
18,43.599,-79.512,20 to 24,<Null>,<Null>,Dark,No Control,Improper Lane Change,"Ability Impaired, Alcohol Over .08","Automobile, Station Wagon",2237.000000
26,43.601,-79.545,50 to 54,Wet,Rain,Dark,Traffic Signal,Following too Close,"Ability Impaired, Alcohol","Automobile, Station Wagon",482.333333
50,43.605,-79.494,75 to 79,Wet,Rain,Daylight,Stop Sign,Lost control,Normal,"Automobile, Station Wagon",843.000000
...,...,...,...,...,...,...,...,...,...,...,...
4634,43.829,-79.280,45 to 49,Wet,Rain,Dusk,Traffic Signal,Failed to Yield Right of Way,Inattentive,"Automobile, Station Wagon",956.666667
4635,43.829,-79.280,70 to 74,Other,Other,Daylight,<Null>,<Null>,<Null>,"Automobile, Station Wagon",956.666667
4649,43.835,-79.182,50 to 54,Wet,Rain,Daylight,No Control,Improper Passing,Normal,"Automobile, Station Wagon",610.000000
4656,43.836,-79.251,unknown,Wet,Rain,"Dark, artificial",Traffic Signal,Failed to Yield Right of Way,Unknown,Municipal Transit Bus (TTC),1187.375000


In [29]:
feature_collapse.to_csv('KSI_Feature_Collapse.csv')